# The Monty Hall Problem

Now that we've had a chance to think about how we might code basic bayesian problems, let's think about how we might apply this to the monty hall problem.

We already know the table approach, but now we want to do this more elegantly. Remember, the key to this exercise is to remember how to calculate the liklihoods and what our hypotheses are.

Remember that we select foor A and the show host opens door B. Given that our hypotheses are about which door the dar is behind, what is the probability that the remaining door has a goat behind it?

* If car behind A, then Monty had even chances of choosing B or C
* If the car is behind B, then we wouldnt be able to open it to show a goat
* If the car is actually behind C then he has to open door B

I'll use the same pandas method I showed before because I am lame

In [1]:
import pandas as pd

In [2]:
# Create list of doors
doors = ['A', 'B', 'C']

# Create dataframe
df_dict = {'doors': doors,
          'prior': [1/len(doors) for x in doors]}
df = pd.DataFrame(df_dict)

df

,doors,prior
0,A,0.333333
1,B,0.333333
2,C,0.333333


In [3]:
# Create liklihood column
df["liklihood"] = pd.Series([0.5, 0, 1])

df

,doors,prior,liklihood
0,A,0.333333,0.5
1,B,0.333333,0.0
2,C,0.333333,1.0


In [4]:
# Multiply prior by the liklihood
df["p_l"] = df.prior * df.liklihood

df

,doors,prior,liklihood,p_l
0,A,0.333333,0.5,0.166667
1,B,0.333333,0.0,0.000000
2,C,0.333333,1.0,0.333333


In [5]:
# Normalise the data
sum_pl = df.p_l.sum()
df["posterior"] = df.p_l / sum_pl

df

,doors,prior,liklihood,p_l,posterior
0,A,0.333333,0.5,0.166667,0.333333
1,B,0.333333,0.0,0.000000,0.000000
2,C,0.333333,1.0,0.333333,0.666667


We can see here that it's best if we switch.

The reason why people get this wrong is because they assume that showing the goat holds no extra information beyond showing that only 1 goat and a car remain. This might even be one of the pitfalls of a naive framework.

The reason why switching has the higher probability is because depending on where the car is the probability he shows a goat is different and that effects the posterior.

In other words, the posteriors would all be equal of monty was choosing at random, he's not!

## Book Solution

Once again we'll write a class just like with the cookie example. It will start off in a similar way...

In [6]:
from thinkbayes2 import Pmf

In [7]:
class Monty(Pmf):
    """
    Creates a class for the monty hall problem.
    """
    def __init__(self, hypos):
        Pmf.__init__(self)  # Once again, initialise the class we're inheriting from
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()
        
    def Update(self, data):
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        self.Normalize()
        
    def Likelihood(self, data, hypo):
        if hypo == data:
            return 0  # i.e. Monty cannot open this door if a car is behind it
        elif hypo == 'A':
            return 0.5  # i.e. If the car is behind A then he could open either B or C
        else:
            return 1 # i.e. If the car is behind C and you chose A then he had to open B

In [8]:
# Create class
hypos = ['A', 'B', 'C']
pmf = Monty(hypos)

In [9]:
# Update data based on observation
data = 'B'
pmf.Update(data)

In [10]:
for hypo, prob in pmf.Items():
    print(hypo, prob)

A 0.3333333333333333
B 0.0
C 0.6666666666666666


Once again we can see the result as given above. 